# Capstone - 2016 Immigration and Temperature Data 

### Data Engineering Capstone Project

#### Project Summary
The goal of this project is to create an ETL pipeline using I94 immigration data and city temperature data to form a database that is optimized for queries on immigration events. This database can be used to answer questions relating immigration behavior to destination temperature e.g., do people tend to immigrate to warmer places?

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
#Used below libraries
import os
from datetime import datetime,timedelta
import pandas as pd, re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import types as T
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import json

### Step 1: Scope the Project and Gather Data

#### Scope 
In this project, we will aggregate I94 immigration data by destination city to form our first dimension table. Next we will aggregate city temperature data by city to form the second dimension table. The two datasets will be joined on destination city to form the fact table. The final database is optimized to query on immigration events to determine if temperature affects the selection of destination cities. Spark will be used to process the data.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The [I94 immigration data](https://travel.trade.gov/research/reports/i94/historical/2016.html) comes from the US National Tourism and Trade Office. It is provided in SAS7BDAT format which is a binary database storage format. Please refer [I94_SAS_Labels_Descriptions.SAS]('I94_SAS_Labels_Descriptions.SAS') document for full reference. Some relevant attributes include:

##### Source Type: __SAS Generated Data__ and __Static Text Files__

* i94yr = 4 digit year
* i94mon = numeric month
* i94cit = 3 digit code of origin city
* i94port = 3 character code of destination USA city
* arrdate = arrival date in the USA
* i94mode = 1 digit travel code
* depdate = departure date from the USA
* i94visa = reason for immigration

The [temperature data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) comes from Kaggle. It is provided in csv format. Some relevant attributes include:

##### Source Type: __CSV Data__

* Dt = date
* AverageTemperature = average temperature
* City = city name
* Country = country name
* Latitude= latitude
* Longitude = longitude

The [U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) data comes from OpenSoft.

##### Source Type: __JSON Data__

* Average_Household_Size
* City
* Count
* Female_Population
* Foreign_Born
* Male_Population
* Median_Age
* Number_of_Veterans
* Race
* State_Code
* Total_Population
* Record_Timestamp

The [Airport Code Table](https://datahub.io/core/airport-codes#data) is a simple table of airport codes and corresponding cities.

##### Source Type: __CSV Data__

* Ident
* Type 
* Name 
* Elevation_Ft 
* Continent 
* Iso_Country
* Iso_Region 
* Municipality 
* Gps_Code 
* Iata_Code 
* Local_Code
* Coordinates


### Step 2: Explore and Assess the Data
#### Explore the Data 
* Read all sources into dataframes and explored the data
* Fixed the Data Type issues and found the PK columns of the data set

##### Immigration Data Analysis:


1. Based on Sample data, assuming the columns will have 100% of data if below the count is 1000 else less than 100%
* cicid     : 1000 Rows
* i94yr     : 1000 Rows
* i94mon    : 1000 Rows
* i94cit    : 1000 Rows
* i94res    : 1000 Rows
* i94port   : 1000 Rows
* arrdate   : 1000 Rows
* i94mode   : 1000 Rows
* i94addr   :  941 Rows
* depdate   :  951 Rows
* i94bir    : 1000 Rows
* i94visa   : 1000 Rows
* count     : 1000 Rows
* dtadfile  : 1000 Rows
* visapost  :  382 Rows
* occup     :    4 Rows
* entdepa   : 1000 Rows
* entdepd   :  954 Rows
* entdepu   :    0 Rows
* matflag   :  954 Rows
* biryear   : 1000 Rows
* dtaddto   : 1000 Rows
* gender    :  859 Rows
* insnum    :   35 Rows
* airline   :  967 Rows
* admnum    : 1000 Rows
* fltno     :  992 Rows
* visatype  : 1000 Rows

After analyzing the file, there are no duplicate records, so keeping all the data.

**Note:** Other files data is stright forward, please see the sample data and analysis in Jupyter cells.

#### Cleaning Steps
* Converted the data types to the right format and dropped the duplicatesif there are any duplicates. Please refer steps below

##### Immigration Data:

1. Converting all non date fields to human readable date format and passing default date as 01/01/1900.
2. Convert float data columns trypes to int.
3. Remove * infront of Airline code.
4. When reading other Text static files into dataframe, clean the signle quotes.

##### US Demographic Data:

1. Converting date in string format to date format.
2. Convert float data columns trypes to int.
3. Extract __fields__ dictionary into notmal fields and remove fields prefix from those.
4. Extract City from **iso_region** 


In [35]:
# Read immigration sample data
df_immigration_pd = pd.read_csv('immigration_data_sample.csv',header=0)
df_immigration_pd = df_immigration_pd.iloc[:,1:]

In [36]:
# Because the immigration data has 28 columns
pd.set_option('display.max_columns', 28)

In [5]:
# Display sample data
df_immigration_pd.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [37]:
#Diplay sample data flights from 209
df_immigration_pd.query('i94port=="LOS"').head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
9,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT
10,1230572.0,2016.0,4.0,438.0,438.0,LOS,20551.0,1.0,CA,20565.0,4.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,2012.0,07052016,F,NaN,QF,5.574381e+10,00015,WT
13,4916639.0,2016.0,4.0,260.0,260.0,LOS,20570.0,1.0,CA,20581.0,62.0,2.0,1.0,20160426,MNL,NaN,G,O,NaN,M,1954.0,10252016,F,NaN,EK,9.461277e+10,00215,B2
27,5070551.0,2016.0,4.0,213.0,213.0,LOS,20571.0,1.0,CA,20676.0,64.0,2.0,1.0,20160427,MDR,NaN,G,O,NaN,M,1952.0,10262016,M,NaN,QR,9.469486e+10,00739,B2


In [7]:
#Verify Data types
df_immigration_pd.dtypes

cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port      object
arrdate     float64
i94mode     float64
i94addr      object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile      int64
visapost     object
occup        object
entdepa      object
entdepd      object
entdepu     float64
matflag      object
biryear     float64
dtaddto      object
gender       object
insnum      float64
airline      object
admnum      float64
fltno        object
visatype     object
dtype: object

In [8]:
# Gives Count of non-NA cells for each column
df_immigration_pd.count()

cicid       1000
i94yr       1000
i94mon      1000
i94cit      1000
i94res      1000
i94port     1000
arrdate     1000
i94mode     1000
i94addr      941
depdate      951
i94bir      1000
i94visa     1000
count       1000
dtadfile    1000
visapost     382
occup          4
entdepa     1000
entdepd      954
entdepu        0
matflag      954
biryear     1000
dtaddto     1000
gender       859
insnum        35
airline      967
admnum      1000
fltno        992
visatype    1000
dtype: int64

In [9]:
#Finding the PK column
len(df_immigration_pd.admnum.unique())

1000

In [ ]:
df_immigration_pd

In [2]:
# Create arrays for each field in i94port codes
i94port_cd = []
i94port_state = []
i94port_city = []
with open('immigration_i94port_valid.txt') as f:
     for line in f:
            line = line.rstrip().lstrip()
            if line:
                #print(line)
                try:
                    key = line.split('=')[0].rstrip().lstrip()[1:-1]
                    val = line.split('=')[1].rstrip().lstrip()[1:-1].split(',')
                    val1 = val[0]
                    val2 = val[1].lstrip().rstrip()
                    i94port_cd.append(key)
                    i94port_state.append(val2)
                    i94port_city.append(val1)
                    
                except IndexError:
                    key = line.split('=')[0].rstrip().lstrip()[1:-1]
                    val = line.split('=')[1].rstrip().lstrip()[1:-1].rstrip()
                    i94port_cd.append(key)
                    i94port_state.append('NaN')
                    i94port_city.append(val)
print('Extracted immigration_i94port_valid.txt file and loaded into Array.')

Extracted immigration_i94port_valid.txt file and loaded into Array.


In [3]:
#reate Pandas DataFrame from arrays and display sample records
i94port_data = {"Port_Code":i94port_cd,"State":i94port_state,"City":i94port_city}
df_i94ports_pd = pd.DataFrame(i94port_data)
df_i94ports_pd.head(5)
print('I94 Port DIM DataFrame Created : df_i94ports_pd')

I94 Port DIM DataFrame Created : df_i94ports_pd


In [12]:
#Display counts
df_i94ports_pd.count()

Port_Code    588
State        588
City         588
dtype: int64

In [13]:
#Counting the bad State records
df_i94ports_pd.query('State == "NaN"')

,Port_Code,State,City
28,MAP,NaN,MARIPOSA AZ
76,WAS,NaN,WASHINGTON DC
572,ZZZ,NaN,MEXICO Land (Banco de Mexico)
573,CHN,NaN,No PORT Code (CHN)
575,MAA,NaN,Abu Dhabi


In [14]:
#Verifying the valid record
df_i94ports_pd.query('Port_Code == "LOS"')

,Port_Code,State,City
44,LOS,CA,LOS ANGELES


In [15]:
df_i94ports_pd.dtypes

Port_Code    object
State        object
City         object
dtype: object

In [4]:
#Reaading i94City file and saving in a DataFrame
df_i94city_pd = pd.read_csv("immigration_i94cit_valid.txt",sep="=", names=["Code","Country"], header=None)
print('Extracted immigration_i94cit_valid.txt into : df_i94city_pd')

Extracted immigration_i94cit_valid.txt into : df_i94city_pd


In [17]:
df_i94city_pd.head(5)

,Code,Country
0,582,"'MEXICO Air Sea, and Not Reported (I-94, no ..."
1,236,'AFGHANISTAN'
2,101,'ALBANIA'
3,316,'ALGERIA'
4,102,'ANDORRA'


In [5]:
#Removing quotes around the Country
df_i94city_pd["Country"] = df_i94city_pd["Country"].apply(lambda x: x.lstrip().rstrip()[1:-1])
df_i94city_pd.dtypes

Code        int64
Country    object
dtype: object

In [19]:
df_i94city_pd.head(5)

,Code,Country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [6]:
df_i94addr_pd = pd.read_csv("immigration_i94addr_valid.txt", sep="=",names =["State_Code","State"],header=None)
df_i94addr_pd.head(5)

,State_Code,State
0,'AL','ALABAMA'
1,'AK','ALASKA'
2,'AZ','ARIZONA'
3,'AR','ARKANSAS'
4,'CA','CALIFORNIA'


In [7]:
#Cleaned single quotes around the data
df_i94addr_pd["State_Code"] = df_i94addr_pd["State_Code"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94addr_pd["State"] = df_i94addr_pd["State"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94addr_pd.head(5)

,State_Code,State
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [22]:
df_i94addr_pd.dtypes

State_Code    object
State         object
dtype: object

In [8]:
#reading and clening VISA DIM data
df_i94visa_pd = pd.read_csv("immigration_i94visa_valid.txt", sep=",",names =["VISA_Code","VISA_Desc"],header=None)
df_i94visa_pd.head()

,VISA_Code,VISA_Desc
0,1,'Business'
1,2,'Pleasure'
2,3,'Student'


In [9]:
#Cleaning single quotes
df_i94visa_pd["VISA_Desc"] = df_i94visa_pd["VISA_Desc"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94visa_pd.head()

,VISA_Code,VISA_Desc
0,1,Business
1,2,Pleasure
2,3,Student


In [11]:
#reading and clening VISA mode data
df_i94mode_pd = pd.read_csv("immigration_i94mode_valid.txt", sep=",",names =["Entry_Code","Entry_Desc"],header=None)
df_i94mode_pd.head()

,Entry_Code,Entry_Desc
0,1,'Air'
1,2,'Sea'
2,3,'Land'
3,9,'Not reported'


In [12]:
#Cleaning single quotes
df_i94mode_pd["Entry_Desc"] = df_i94mode_pd["Entry_Desc"].apply(lambda x : x.lstrip().rstrip()[1:-1])
df_i94mode_pd.head()

,Entry_Code,Entry_Desc
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [27]:
df_i94mode_pd.dtypes

Entry_Code     int64
Entry_Desc    object
dtype: object

In [13]:
#Reading Global Weather Data
df_global_temp = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv',header=0)
df_global_temp.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [29]:
#Finding Los Angeles record
df_global_temp.query('City == "Los Angeles" and Country == "United States"').head(2)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
4356748,1849-01-01,8.819,2.558,Los Angeles,United States,34.56N,118.70W
4356749,1849-02-01,9.577,1.970,Los Angeles,United States,34.56N,118.70W


In [30]:
#Verifying Date format
df_global_temp.dt.apply(lambda x : x.find("/")).head(1)

0   -1
Name: dt, dtype: int64

In [14]:
#Converting dt from string to date data type
df_global_temp['dt'] =  pd.to_datetime(df_global_temp['dt'])
df_global_temp.dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
City                                     object
Country                                  object
Latitude                                 object
Longitude                                object
dtype: object

In [15]:
df_global_temp.head(3)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [33]:
df_global_temp.count()

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64

In [16]:
#Reading US demographics data and printing data types
with open('us-cities-demographics.json','r') as f:
    data = json.load(f)
df_us_info = pd.io.json.json_normalize(data)
df_us_info.dtypes

datasetid                         object
fields.average_household_size    float64
fields.city                       object
fields.count                       int64
fields.female_population         float64
fields.foreign_born              float64
fields.male_population           float64
fields.median_age                float64
fields.number_of_veterans        float64
fields.race                       object
fields.state                      object
fields.state_code                 object
fields.total_population            int64
record_timestamp                  object
recordid                          object
dtype: object

In [35]:
#Sample Data
df_us_info.head(3)

,datasetid,fields.average_household_size,fields.city,fields.count,fields.female_population,fields.foreign_born,fields.male_population,fields.median_age,fields.number_of_veterans,fields.race,fields.state,fields.state_code,fields.total_population,record_timestamp,recordid
0,us-cities-demographics,2.73,Newark,76402,143873.0,86253.0,138040.0,34.6,5829.0,White,New Jersey,NJ,281913,1969-12-31T19:00:00-05:00,85458783ecf5da6572ee00e7120f68eff4fd0d61
1,us-cities-demographics,2.40,Peoria,1343,62432.0,7517.0,56229.0,33.1,6634.0,American Indian and Alaska Native,Illinois,IL,118661,1969-12-31T19:00:00-05:00,a5ad84bdb4d72688fb6ae19a8bee43bcb01f9fea
2,us-cities-demographics,2.77,O'Fallon,2583,43270.0,3269.0,41762.0,36.0,5783.0,Hispanic or Latino,Missouri,MO,85032,1969-12-31T19:00:00-05:00,c54cd5021a16eb5f7b83987742bd495229b2155e


In [17]:
#Renaming the columns
df_us_info.set_axis(['datasetid','average_household_size','city','count','female_population',
                     'foreign_born','male_population','median_age','number_of_veterans','race',
                     'state','state_code','total_population','record_timestamp','recordid'], 
                    axis=1, inplace=True)

In [25]:
#Converting to datetime format and querying sample data
df_us_info['record_timestamp'] =  pd.to_datetime(df_us_info['record_timestamp'])
df_us_info.query('state_code=="CA" and city == "Los Angeles"')

,datasetid,average_household_size,city,count,female_population,foreign_born,male_population,median_age,number_of_veterans,race,state,state_code,total_population,record_timestamp,recordid
97,us-cities-demographics,2.86,Los Angeles,2177650,2012898.0,1485425.0,1958998.0,35.0,85417.0,White,California,CA,3971896,1970-01-01,7da42fda61238faccac3d43954a8f621a3a51194
554,us-cities-demographics,2.86,Los Angeles,512999,2012898.0,1485425.0,1958998.0,35.0,85417.0,Asian,California,CA,3971896,1970-01-01,e23be85ef2bf6caecf2309ba6dedc868929d1377
729,us-cities-demographics,2.86,Los Angeles,404868,2012898.0,1485425.0,1958998.0,35.0,85417.0,Black or African-American,California,CA,3971896,1970-01-01,cda8c0b63e4c14d174940e1df5d50d2d2491ccfa
1225,us-cities-demographics,2.86,Los Angeles,63758,2012898.0,1485425.0,1958998.0,35.0,85417.0,American Indian and Alaska Native,California,CA,3971896,1970-01-01,f45da4b5c979eb53a8e10a5d4e6ef2a7bb480fbc
1899,us-cities-demographics,2.86,Los Angeles,1936732,2012898.0,1485425.0,1958998.0,35.0,85417.0,Hispanic or Latino,California,CA,3971896,1970-01-01,5212831e25cadd6f383d1bf93274aa17e346adb6


In [28]:
df_us_info.record_timestamp.dt.year.unique()

array([1970])

In [19]:
#Verifying Null values
df_us_info.count()

datasetid                 2891
average_household_size    2875
city                      2891
count                     2891
female_population         2888
foreign_born              2878
male_population           2888
median_age                2891
number_of_veterans        2878
race                      2891
state                     2891
state_code                2891
total_population          2891
record_timestamp          2891
recordid                  2891
dtype: int64

In [29]:
#Reading Airport Codes Data
df_airport_cd = pd.read_csv('airport-codes_csv.csv',header=0)

In [40]:
#Checking iso_region format
df_airport_cd[df_airport_cd.iso_region.str.len()>5].head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
174,02PR,small_airport,Cuylers Airport,15.0,NaN,PR,PR-U-A,Vega Baja,02PR,NaN,02PR,"-66.36689758300781, 18.45330047607422"
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222"


In [30]:
#extracting city from region field
df_airport_cd['city'] = df_airport_cd['iso_region'].apply(lambda x: x[x.find('-')+1:])

In [22]:
#Checking iso_region format after cleaning
df_airport_cd[df_airport_cd.iso_region.str.len()>5].head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
174,02PR,small_airport,Cuylers Airport,15.0,NaN,PR,PR-U-A,Vega Baja,02PR,NaN,02PR,"-66.36689758300781, 18.45330047607422",U-A
223,03N,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,K03N,UTK,03N,"169.852005, 11.222",UTI


In [33]:
df_airport_cd.query('iso_country == "US" and municipality=="Los Angeles"').type.unique()

array(['heliport', 'closed', 'large_airport', 'small_airport'], dtype=object)

In [34]:
df_airport_cd.query('iso_country == "US" and municipality=="Los Angeles" and type=="large_airport"')

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
27795,KLAX,large_airport,Los Angeles International Airport,125.0,NaN,US,US-CA,Los Angeles,KLAX,LAX,LAX,"-118.4079971, 33.94250107",CA


In [44]:
df_airport_cd.count()

ident           55075
type            55075
name            55075
elevation_ft    48069
continent       27356
iso_country     54828
iso_region      55075
municipality    49399
gps_code        41030
iata_code        9189
local_code      28686
coordinates     55075
city            55075
dtype: int64

In [45]:
#Creating dates for DATE_DIM
import numpy as np
df_date_dim = pd.DataFrame(pd.period_range('1/1/2015', freq='D', periods=2000),columns=['date_id'], dtype='datetime64[ns]')

In [46]:
#verifying the dates
df_date_dim.head(5)

,date_id
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [47]:
#Adding other fields
df_date_dim['year'],df_date_dim['month'],df_date_dim['day'],df_date_dim['dayofweek'],df_date_dim['weekofyear'] =\
df_date_dim.date_id.dt.year,df_date_dim.date_id.dt.month,df_date_dim.date_id.dt.day,df_date_dim.date_id.dt.dayofweek,df_date_dim.date_id.dt.weekofyear

In [48]:
#sampling the data
df_date_dim.sample(5)

,date_id,year,month,day,dayofweek,weekofyear
1926,2020-04-10,2020,4,10,4,15
1494,2019-02-03,2019,2,3,6,5
550,2016-07-04,2016,7,4,0,27
1282,2018-07-06,2018,7,6,4,27
1418,2018-11-19,2018,11,19,0,47


In [49]:
#verifying the data types
df_date_dim.dtypes

date_id       datetime64[ns]
year                   int64
month                  int64
day                    int64
dayofweek              int64
weekofyear             int64
dtype: object

In [50]:
#Creating Spark session with sas data
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [51]:
%%time
#Reading sample SAS immigration data set into DataFrame
df_immigration_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

CPU times: user 1.56 ms, sys: 227 µs, total: 1.79 ms
Wall time: 2.79 s


In [52]:
%%time
#Total records
df_immigration_spark.count()

CPU times: user 6.8 ms, sys: 989 µs, total: 7.78 ms
Wall time: 43.1 s


3096313

In [53]:
%%time
#Checking for the duplicate records and matching with total records
df_immigration_spark.distinct().count()

CPU times: user 5.16 ms, sys: 8.09 ms, total: 13.3 ms
Wall time: 1min 14s


3096313

In [54]:
%%time
#finding the PK column
df_immigration_spark.select('admnum').distinct().count()
print('Admnum can be  a PK')

Admnum can be  a PK
CPU times: user 5.45 ms, sys: 4.76 ms, total: 10.2 ms
Wall time: 43.2 s


In [55]:
#sample data
df_immigration_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [56]:
#user defined functions to convert the dates and remove * from data
def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return datetime(1900,1,1) 
udf_datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())

def convert_YYYYMMDD_datetime(x):
    try:
        x = str(x)
        return datetime(int(x[0:4]),int(x[4:6]),int(x[6:8]))

    except:
        return datetime(1900,1,1) 
udf_datetime_from_YYYYMMDD = udf(lambda x: convert_YYYYMMDD_datetime(x), T.DateType())

def convert_MMDDYYYY_datetime(x):
    try:
        x = str(x)
        return datetime(int(x[4:8]),int(x[0:2]),int(x[2:4]))
    except:
        return datetime(1900,1,1) 
udf_datetime_from_MMDDYYYY = udf(lambda x: convert_MMDDYYYY_datetime(x), T.DateType())

def remove_Special_Char(x):
    try:
        return x.lstrip().rstrip().lstrip('*').rstrip('*')
    except:
        return '-'
    
udf_remove_Special_Char = udf(lambda x: remove_Special_Char(x),T.StringType())
print('Created 3 user defined functions to convert dates.')
print('Created 1 user defined function to remove * and spaces around the airline column.')

Created 3 user defined functions to convert dates.
Created 1 user defined function to remove * and spaces around the airline column.


In [57]:
#Calling udf to convert arrival date
df_immigration_spark = df_immigration_spark.withColumn("Arrival_Date",udf_datetime_from_sas(df_immigration_spark.arrdate))

In [58]:
df_immigration_spark.where(df_immigration_spark.Arrival_Date.isNull()).show(5)

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender|insnum|airline|admnum|fltno|visatype|Arrival_Date|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+



In [59]:
df_immigration_spark = df_immigration_spark.withColumn("I94_added_date",udf_datetime_from_YYYYMMDD(df_immigration_spark.dtadfile))\
.withColumn("VISA_Expiry_date",udf_datetime_from_MMDDYYYY(df_immigration_spark.dtaddto))

In [60]:
df_immigration_spark.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+----------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|Arrival_Date|I94_added_date|VISA_Expiry_date|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+------------+--------------+----------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.89762

In [61]:
#Converting all decimals to integer
df_immigration_spark = df_immigration_spark.withColumn("i94yr",df_immigration_spark.i94yr.cast('int'))\
.withColumn("i94mon",df_immigration_spark.i94mon.cast('int'))\
.withColumn("i94cit",df_immigration_spark.i94cit.cast('int'))\
.withColumn("i94res",df_immigration_spark.i94res.cast('int'))\
.withColumn("arrdate",df_immigration_spark.arrdate.cast('int'))\
.withColumn("i94mode",df_immigration_spark.i94mode.cast('int'))\
.withColumn("depdate",df_immigration_spark.depdate.cast('int'))\
.withColumn("i94bir",df_immigration_spark.i94bir.cast('int'))\
.withColumn("dtadfile",df_immigration_spark.dtadfile.cast('int'))\
.withColumn("biryear",df_immigration_spark.biryear.cast('int'))\
.withColumn("admnum",df_immigration_spark.admnum.cast('int'))\
.withColumn("i94visa",df_immigration_spark.i94visa.cast('int'))


In [62]:
#converting dates
df_immigration_spark = df_immigration_spark.withColumn("arrdate",udf_datetime_from_sas(df_immigration_spark.arrdate))\
.withColumn("depdate",udf_datetime_from_sas(df_immigration_spark.depdate))\
.withColumn("dtadfile",udf_datetime_from_YYYYMMDD(df_immigration_spark.dtadfile))\
.withColumn("dtaddto",udf_datetime_from_MMDDYYYY(df_immigration_spark.dtaddto))\
.withColumn("airline",udf_remove_Special_Char(df_immigration_spark.airline))

In [63]:
%%time
#Top 4 records where cicid=610
df_immigration_spark.filter(df_immigration_spark.cicid==610).show(4)

+-----+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+----------+--------+-----+-------+-------+-------+-------+-------+----------+------+------+-------+----------+-----+--------+------------+--------------+----------------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir|i94visa|count|  dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|   dtaddto|gender|insnum|airline|    admnum|fltno|visatype|Arrival_Date|I94_added_date|VISA_Expiry_date|
+-----+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+----------+--------+-----+-------+-------+-------+-------+-------+----------+------+------+-------+----------+-----+--------+------------+--------------+----------------+
|610.0| 2016|     4|   103|   103|    LVG|2016-04-01|      1|     NV|2016-04-02|    58|      2|  1.0|2016-04-01|     BRL| null|      G|      R|   null|      M|   1958|2016-09-30|     M

In [72]:
#Reading SAS source file names
src_sas_path='../../data/18-83510-I94-Data-2016'
sas_source_files =[]
for i in os.listdir(src_sas_path):
    sas_source_files.append(os.path.join(src_sas_path,i))

In [73]:
#Printing the source file names
for i in sas_source_files:
    print(i)

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat
../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat


In [74]:
#Keeping only 2 sample files in sas_source_files list 
sas_source_files = sas_source_files[0:1]

In [77]:
sas_source_files

['../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat']

In [76]:
%%time
#1. Converting all non date fields to human readable date format.
#2. Convert float data columns trypes to int.
#3. Remove * infront of Airline code.
#4. When reading other Text static files into dataframe, clean the signle quotes.

def read_sas_files(file_path):
    file_path = str(file_path)
    #extracting month and year part from the file
    tgt_path = file_path[file_path.find('/i94_')+5:file_path.find('_sub')]
    
    #converting all float fields to Number format
    df_spark = spark.read.format('com.github.saurfang.sas.spark').load(file_path)
    df_spark = df_spark.withColumn("i94yr",df_spark.i94yr.cast('int'))\
      .withColumn("i94mon",df_spark.i94mon.cast('int'))\
      .withColumn("i94cit",df_spark.i94cit.cast('int'))\
      .withColumn("i94res",df_spark.i94res.cast('int'))\
      .withColumn("arrdate",df_spark.arrdate.cast('int'))\
      .withColumn("i94mode",df_spark.i94mode.cast('int'))\
      .withColumn("depdate",df_spark.depdate.cast('int'))\
      .withColumn("i94bir",df_spark.i94bir.cast('int'))\
      .withColumn("dtadfile",df_spark.dtadfile.cast('int'))\
      .withColumn("biryear",df_spark.biryear.cast('int'))\
      .withColumn("admnum",df_spark.admnum.cast('int'))\
      .withColumn("i94visa",df_spark.i94visa.cast('int'))
    
    #Converting string to date format
    df_spark = df_spark.withColumn("arrdate",udf_datetime_from_sas(df_spark.arrdate))\
      .withColumn("depdate",udf_datetime_from_sas(df_spark.depdate))\
      .withColumn("dtadfile",udf_datetime_from_YYYYMMDD(df_spark.dtadfile))\
      .withColumn("dtaddto",udf_datetime_from_MMDDYYYY(df_spark.dtaddto))\
      .withColumn("airline",udf_remove_Special_Char(df_spark.airline))
    
    df_spark.write.parquet("results/"+tgt_path)
    return [file_path,df_spark]

#using Concurrent futures to execute the reading and writing the files in Parallel.
#This work only if the server has CPU and Memory capabilities to process the tasks in parallel.

max_workers = len(sas_source_files)
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_count = {executor.submit(read_sas_files,file_path) : file_path for file_path in sas_source_files}
    for record in concurrent.futures.as_completed(future_count):
        print('File '+record.result()[0]+' process completed...')
        #print(record.result()[1].show(2))

File ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat process completed...
CPU times: user 30.2 ms, sys: 966 µs, total: 31.2 ms
Wall time: 1min 59s


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Based on the Source data, prepared the below conceptual model and you can derive many solutions based on this data

<img src="Immigration and Temperature DB Logical Design.jpg" >


#### 3.2 Mapping Out Data Pipelines

##### Steps:
**General Solution (Local Machine):**
* Extract source files into DataFrame.
* Transform/Cleanup the data in the DataFrame.
* Load all DataFrames into corresponding **Parquet** files.
* Analyze the data.

**Cloud Solution (AWS EMR):**
* Create Tables in Cloud Database.
* Extract source data into DataFrames.
* Transform/Cleanup the data in the DataFrame.
* Load the data into DIM Tables.
* Load the Fact table.
* Analyze the data.

Note: This is kind of Data Lake, there is no order in reading/writing. Only when you anlayze the quries, you need the data in all tables.

<img src="Data Flow Diagram.jpg" />

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

##### Physical Data Model (Star Schema):

###### Dimention Tables:
<p style = "font-size:12px">
CREATE TABLE AIRPORTS_CODES_DIM(<br>
IDENTIFICATION VARCHAR(10) NOT NULL,<br>
NAME VARCHAR(100),<br>
TYPE VARCHAR(30),<br>
CONTINENT VARCHAR(10),<br>
COUNTRY VARCHAR(10),<br>
CITY VARCHAR(10),<br>
MUNICIPALITY VARCHAR(50),<br>
PRIMARY KEY(IDENTIFICATION))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE US_CITY_DEMOGRAPHIC_DIM(<br>
ID INT NOT NULL,<br>
STATE VARCHAR(10),<br>
CITY VARCHAR(50),<br>
RECORD_TIMESTAMP TIMESTAMP,<br>
TOTAL_POPULATION  BIGINT,<br>
MALE_POPULATION BIGINT,<br>
FEMALE_POPULATION BIGINT,<br>
COUNT BIGINT,<br>
MEDIAN_AGE  SMALLINT,<br>
AVERAGE_HOUSEHOLD_SIZE SMALLINT,<br>
RACE  VARCHAR(50),<br>
FOREIGN_BORN BIGINT,<br>
NUMBER_OF_VETERANS  SMALLINT,<br>
PRIMARY KEY(ID))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_PORT_DIM(<br>
PORT_CODE VARCHAR(5),<br>
STATE VARCHAR(5),<br>
CITY VARCHAR(30),<br>
PRIMARY KEY(PORT_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_ENTRY_MODE_DIM(<br>
ENTRY_CODE SMALLINT NOT NULL,<br>
ENTRY_DESC VARCHAR(15) NOT NULL,<br>
PRIMARY KEY(ENTRY_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_ADDRESS_DIM(<br>
STATE_CODE  CHAR(2) NOT NULL,<br>
STATE VARCHAR(30),<br>
PRIMARY KEY(STATE_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_VISA_DIM(<br>
VISA_CODE SMALLINT NOT NULL,<br>
VISA_DESC VARCHAR(20) NOT NULL,<br>
PRIMARY KEY(VISA_CODE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE DATE_DIM(<br>
DATE DATE NOT NULL,<br>
DAY  SMALLINT,<br>
MONTH SMALLINT,<br>
YEAR INTEGER<br>
DAY_OF_WEEK SMALLINT,<br>
WEEK_OF_YEAR SMALLINT,<br>
PRIMARY KEY(DATE))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE GLOBAL_WEATHER_DIM(<br>
ID SMALLINT NOT NULL,<br>
DATE DATE,<br>
CITY VARCHAR(70),<br>
COUNTRY VARCHAR(30),<br>
AVERAGE_TEMPERATURE SMALLINT,<br>
AVERAGE_TEMPERATURE_UNCERTAIN SMALLINT,<br>
LATITUDE SMALLINT,<br>
LONGITUDE SMALLINT,<br>
PRIMARY KEY(ID))<br>
DISTSTYLE ALL;<br>
<br>
CREATE TABLE 194_CIT_RES_DIM(<br>
CODE SMALLINT NOT NULL,<br>
COUNTRY VARCHAR(50),<br>
PRIMARY KEY(CODE))<br>
DISTSTYLE ALL;</p>
<br>

###### Fact Table:
<p style = "font-size:12px">
CREATE TABLE IF NOT EXISTS 194_IMMIGRATION_FACT(<br>
ADMISSION_NUMBER INTEGER NOT NULL,<br>
YEAR YEAR NOT NULL,<br>
MONTH MONTH NOT NULL,<br>
I94_CITY  SMALLINT NOT NULL,<br>
I94_RES  SMALLINT NOT NULL,<br>
PORT  VARCHAR(10),<br>
ARRIVAL_DATE DATE NOT NULL,<br>
ENTRY_MODE SMALLINT,<br>
ARRIVAL_STATE VARCHAR(5),<br>
AGE SMALLINT,<br>
VISA_CODE SMALLINT,<br>
VISA_POST VARCHAR(20),<br>
OCCUPATION VARCHAR(50),<br>
ARRIVAL_FLAG CHAR(1),<br>
DEPARTURE_FLAG CHAR(1),<br>
UPDATE_FLAG CHAR(1),<br>
MATCH_FLAG CHAR(1),<br>
VISA_EXPIRE_DATE DATE NOT NULL,<br>
GENDER CHAR(1),<br>
AIRLINE VARCHAR(5),<br>
FLIGHT_NUMBER VARCHAR(10),<br>
VISA_TYPE VARCHAR(10),<br>
PRIMARY KEY(ADMISSION_NUMBER))<br>
);<br>
<br>
</p>

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [39]:
#Creating Spark session with sas data
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [40]:
#Reading all Parquet files
df_spark_par=spark.read.parquet("results/*/")

In [41]:
%%time
#record count
df_spark_par.count()

CPU times: user 0 ns, sys: 1.58 ms, total: 1.58 ms
Wall time: 2.66 s


3096313

In [82]:
%%time
#distinct records
df_spark_par.distinct().count()

CPU times: user 0 ns, sys: 5.05 ms, total: 5.05 ms
Wall time: 38.6 s


3096313

In [85]:
#give me the largest airport name most people travelled in 2016 Apr
df_pd_i94port_top3 = df_spark_par.groupBy('i94port').count().sort('count',ascending=False).toPandas().head(3)
df_pd_i94port_top3.head()

,i94port,count
0,NYC,485916
1,MIA,343941
2,LOS,310163


In [86]:
df_pd_i94port_top3.merge(df_airport_cd,left_on='i94port',right_on='local_code')

,i94port,count,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
0,MIA,343941,KMIA,large_airport,Miami International Airport,8.0,NaN,US,US-FL,Miami,KMIA,MIA,MIA,"-80.29060363769531, 25.79319953918457",FL
1,LOS,310163,MX-0438,small_airport,Los Charcos Airstrip,8879.0,NaN,MX,MX-DUR,Mezquital,NaN,NaN,LOS,"-104.293973, 23.000115",DUR


In [49]:
df_airport_cd.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125",PA
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022",KS
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968",AK
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172",AL
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087",AR


In [52]:
df_airport_cd.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [92]:
#df_airport_cd.query('type == "seaplane_base"').head(3)
df_airport_cd.query('local_code == "NYC"').head(3)
df_airport_cd.query('type == "large_airport" and city=="NY"')

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city
26279,KBUF,large_airport,Buffalo Niagara International Airport,728.0,NaN,US,US-NY,Buffalo,KBUF,BUF,BUF,"-78.73220062, 42.94049835",NY
27679,KJFK,large_airport,John F Kennedy International Airport,13.0,NaN,US,US-NY,New York,KJFK,JFK,JFK,"-73.77890015, 40.63980103",NY
27819,KLGA,large_airport,La Guardia Airport,21.0,NaN,US,US-NY,New York,KLGA,LGA,LGA,"-73.87259674, 40.77719879",NY
29806,KROC,large_airport,Greater Rochester International Airport,559.0,NaN,US,US-NY,Rochester,KROC,ROC,ROC,"-77.67240142822266, 43.118900299072266",NY
30055,KSYR,large_airport,Syracuse Hancock International Airport,421.0,NaN,US,US-NY,Syracuse,KSYR,SYR,SYR,"-76.1063003540039, 43.11119842529297",NY
49898,US-0883,large_airport,JFK,NaN,NaN,US,US-NY,New York,NaN,NaN,NaN,"0, 0",NY
50032,US-1016,large_airport,JFK,NaN,NaN,US,US-NY,New York City,NaN,NaN,NaN,"0, 0",NY


In [57]:
df_i94ports_pd.head(5)

,Port_Code,State,City
0,ALC,AK,ALCAN
1,ANC,AK,ANCHORAGE
2,BAR,AK,BAKER AAF - BAKER ISLAND
3,DAC,AK,DALTONS CACHE
4,PIZ,AK,DEW STATION PT LAY DEW


In [62]:
df_airport_cd.query('type != "large_airport"').merge(df_i94ports_pd,left_on='local_code',right_on='Port_Code').head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,city,Port_Code,State,City
0,48Y,small_airport,Piney Pinecreek Border Airport,1078.0,NaN,US,US-MN,Pinecreek,48Y,NaN,48Y,"-95.98259735107422, 48.99959945678711",MN,48Y,MN,PINECREEK BORDER ARPT
1,5KE,seaplane_base,Ketchikan Harbor Seaplane Base,NaN,NaN,US,US-AK,Ketchikan,NaN,WFB,5KE,"-131.677002, 55.349899",AK,5KE,AK,KETCHIKAN
2,AEK,small_airport,Aseki Airport,4106.0,OC,PG,PG-MPL,Aseki,AYAX,AEK,ASE,"146.19386673, -7.35080485552",MPL,ASE,CO #ARPT,ASPEN
3,KASE,medium_airport,Aspen-Pitkin Co/Sardy Field,7820.0,NaN,US,US-CO,Aspen,KASE,ASE,ASE,"-106.8690033, 39.22320175",CO,ASE,CO #ARPT,ASPEN
4,AIH,small_airport,Aiambak Airport,90.0,OC,PG,PG-WPD,Aiambak,AYAK,AIH,AMB,"141.2675, -7.342777777779999",WPD,AMB,ND,AMBROSE


In [ ]:
%%time
#Grouping By VISATYPE=B2 and sort in descending order
df_spark_par.where(df_spark_par.visatype=="B2").groupBy(df_spark_par.i94mon).count().sort("count",ascending=False).collect()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.